# ECON30076 Industrial Economics 
## Week 4 - Assessing Market Power

Code for the live lecture

This code first simulates choice data given some parameters and then 
estimates the demand parameters in a Logit model.

Outline of the code:

0. Settings for the code
1. Simulate choice data
2. Estimate Logit model to recover demand parameters

In [12]:
## First, import the required packages
# Pandas to work with DataFrames
import pandas as pd
# Numpy for random draws
import numpy as np

### 0. Settings

We assume consumers choose one out of three PCs. Let us say consumers only
care about the price and the number of processing cores.

Consumers choose according to indirect utility

u = beta_p * price + beta_cores * cores + epsilon

where epsilon follows an Extreme value type I distribution.

For the simulation, we need to set values for beta_p and beta_cores
and we will then draw from the Extreme value type I distribution to simulate
choices.


In [13]:
### Set values for the utility coefficients
beta_p = -10
beta_cores = 10

### Setting a seed allows you to reproduce results involving random numbers
# remove this to obtain different results every time you run it
np.random.seed(1234)

### 1. Simulate data

Steps:
1. Create empty data set for 50 consumers choosing from 3 goods
    you can change these numbers if you like
2. Create observable data (price, cores) for three products
3. Calculate mean utility for each observation
4. Draw from Extreme value distribution for each observation
5. Indicate choice for each consumer that maximises indirect utility

Note: When writing a demand estimation code, it is generally good practice to 
simulate data and try to recover your parameters. This is a good way to check 
your estimation code works correctly.

In [14]:
# 1. Create empty data set for 50 consumers choosing from 3 goods
#     you can change these numbers if you like
n_cons = 50
n_prods = 3

In [15]:
###### 1.1 Create basic data set structure

### We need two identifiers: one for the products, one for the consumers
# 50 consumers, each considers all three products

# Create a vector of consumer identifiers
id_cons = [x for x in range(n_cons)]

## Loop through products and create a DataFrame with these consumer IDs and 
# the respective product ID. Add everything into a list
# Empty list to save in
dflist = []
for i in range(n_prods):
    df_sub = pd.DataFrame(id_cons, columns = ['consumer_id'])
    # Add column with product ID
    df_sub['product_id'] = i
    # Add to list
    dflist.append(df_sub)
    
## Combine dflist to one DataFrame
df = pd.concat(dflist)
# Sort by consumer_id first
df.sort_values(['consumer_id', 'product_id'], inplace = True)
# Reset index
df.reset_index(drop = True, inplace = True)

In [16]:
###### 1.2 Create product characteristics
# Draw prices and cores from uniform distribution between 0 and 1
df['price'] = np.random.uniform(size = len(df))
df['cores'] = np.random.uniform(size = len(df))

In [17]:
###### 1.3 Calculate mean utility
# Given the observable product characteristics and the beta coefficients from
# above, we can calculate the mean utility (denoted as delta in the slides)
df['delta'] = beta_p * df['price'] + beta_cores * df['cores']

In [18]:
###### 1.4 Draw error terms
### We draw the unobserved utility component from an Extreme value distribution 
# The standard Extreme value distribution is also called Standard Gumbel distribution

# Draw Standard Gumbel distribution using numpy
df['errors'] = np.random.gumbel(size = len(df))

In [19]:
###### 1.5 Simulate choices
### Calculate indirect utility as delta + errors
df['utility'] = df['delta'] + df['errors']

### For each consumer, take maximum utility value
df_max = df[['consumer_id', 'utility']].groupby('consumer_id').agg('max') \
    .reset_index().rename(columns = {'utility': 'maxU'})
# Merge back
df = df.merge(df_max, on = 'consumer_id', how = 'left', validate = 'm:1')
# Indicator for chosen product
df['chosen'] = (df['utility'] == df['maxU']).astype(int)

### Look at choices
df.groupby(['product_id', 'chosen']).size()

product_id  chosen
0           0         28
            1         22
1           0         40
            1         10
2           0         32
            1         18
dtype: int64

### 2. Estimate logit model

In [20]:
# Before estimating, let us 'forget' everything that would be normally unobserved.
df.drop(columns = ['delta', 'errors', 'utility', 'maxU'], inplace = True)

In [21]:
###### 2.1 Estimation by hand

#### Start by assuming some value for beta_p and beta_cores
beta_p_ml = 0
beta_cores_ml = 0
# Use these to calculate log likelihood

### Recall: Individual likelihoods are given by logit choice probability
# prob = exp(delta)/sum of exp(delta) over all choices
# delta = beta_p_ml * price + beta_cores_ml * cores

# Delta
df['delta'] = beta_p_ml * df['price'] + beta_cores_ml * df['cores']
# Calculate exp(delta)
df['exp_d'] = np.exp(df['delta'])
# Calculate sum of exps of delta for each consumer
df_cons = df[['consumer_id', 'exp_d']].groupby('consumer_id').agg('sum') \
    .reset_index().rename(columns = {'exp_d': 'sum_exp'})
df = df.merge(df_cons, on = 'consumer_id', how = 'left', validate = 'm:1')
# Calculate logit choice probability
df['probs'] = df['exp_d'] / df['sum_exp']
# Take log likelihood
df['ll'] = np.log(df['probs'])

### For the total log likelihood, we want to sum individual likelihoods, but only
# for the chosen alternatives
df['ll_chosen'] = df['ll'] * df['chosen']
# Sum and display total log likelihood
ll = df['ll_chosen'].sum()
print('\nbeta_p = %s, beta_cores = %s\nTotal log-likelihood: %s' \
    % (beta_p_ml, beta_cores_ml, ll))


beta_p = 0, beta_cores = 0
Total log-likelihood: -54.93061443340549


In [22]:
#### Let us try again, this time with the true parameter values
beta_p_ml = beta_p
beta_cores_ml = beta_cores
# Drop previously created variables
df.drop(columns = ['delta', 'exp_d', 'sum_exp', 'probs', 'll'], inplace = True)

# Delta
df['delta'] = beta_p_ml * df['price'] + beta_cores_ml * df['cores']
# Calculate exp(delta)
df['exp_d'] = np.exp(df['delta'])
# Calculate sum of exps of delta for each consumer
df_cons = df[['consumer_id', 'exp_d']].groupby('consumer_id').agg('sum') \
    .reset_index().rename(columns = {'exp_d': 'sum_exp'})
df = df.merge(df_cons, on = 'consumer_id', how = 'left', validate = 'm:1')
# Calculate logit choice probability
df['probs'] = df['exp_d'] / df['sum_exp']
# Take log likelihood
df['ll'] = np.log(df['probs'])

### For the total log likelihood, we want to sum individual likelihoods, but only
# for the chosen alternatives
df['ll_chosen'] = df['ll'] * df['chosen']
# Sum and display total log likelihood
ll = df['ll_chosen'].sum()
print('\nbeta_p = %s, beta_cores = %s\nTotal log-likelihood: %s' \
    % (beta_p_ml, beta_cores_ml, ll))


beta_p = -10, beta_cores = 10
Total log-likelihood: -15.369275992416942


In [23]:
#### Let us try again, this time with some other parameter values
beta_p_ml = -1
beta_cores_ml = 1
# Drop previously created variables
df.drop(columns = ['delta', 'exp_d', 'sum_exp', 'probs', 'll'], inplace = True)

# Delta
df['delta'] = beta_p_ml * df['price'] + beta_cores_ml * df['cores']
# Calculate exp(delta)
df['exp_d'] = np.exp(df['delta'])
# Calculate sum of exps of delta for each consumer
df_cons = df[['consumer_id', 'exp_d']].groupby('consumer_id').agg('sum') \
    .reset_index().rename(columns = {'exp_d': 'sum_exp'})
df = df.merge(df_cons, on = 'consumer_id', how = 'left', validate = 'm:1')
# Calculate logit choice probability
df['probs'] = df['exp_d'] / df['sum_exp']
# Take log likelihood
df['ll'] = np.log(df['probs'])

### For the total log likelihood, we want to sum individual likelihoods, but only
# for the chosen alternatives
df['ll_chosen'] = df['ll'] * df['chosen']
# Sum and display total log likelihood
ll = df['ll_chosen'].sum()
print('\nbeta_p = %s, beta_cores = %s\nTotal log-likelihood: %s' \
    % (beta_p_ml, beta_cores_ml, ll))


beta_p = -1, beta_cores = 1
Total log-likelihood: -41.82032982257603


Now, we would need a way to loop over various combinations of parameter
values to find the ones that maximise the log-likelihood
Luckily, people have implemented packages that automate this for us already

In [24]:
#%% 2.2 Estimation using pylogit package
import pylogit
from collections import OrderedDict

### Prepare specification
basic_specification = OrderedDict()
basic_names = OrderedDict()

for var in ['price', 'cores']:
    
    # Add variable to ordered dict
    basic_specification[var] = [list(set(df["product_id"]))]
    basic_names[var] = [var]
    
### Create the multinomial logit model (MNL) estimator
est_mnl = pylogit.create_choice_model(data = df,
    alt_id_col = "product_id",
    obs_id_col = "consumer_id",
    choice_col = "chosen",
    specification = basic_specification,
    model_type = "MNL",
    names = basic_names)

# Specify the initial values and method for the optimization and estimate
est_mnl.fit_mle(np.zeros(len(basic_specification)))

# Look at the estimation results
print(est_mnl.get_statsmodels_summary())

Log-likelihood at zero: -54.9306
Initial Log-likelihood: -54.9306
Estimation Time for Point Estimation: 0.00 seconds.
Final log-likelihood: -15.0864


/opt/miniconda3/envs/ugio/lib/python3.9/site-packages/pylogit/estimation.py:678: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      chosen   No. Observations:                   50
Model:             Multinomial Logit Model   Df Residuals:                       48
Method:                                MLE   Df Model:                            2
Date:                     Tue, 10 Feb 2026   Pseudo R-squ.:                   0.725
Time:                             13:10:55   Pseudo R-bar-squ.:               0.689
AIC:                                34.173   Log-Likelihood:                -15.086
BIC:                                37.997   LL-Null:                       -54.931
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
price        -10.5906      2.656     -3.987      0.000     -15.796      -5.385
cores         11.7723      3.303      3.565      0.000       5.299      18.245
